In [16]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import os
import re

In [17]:
file_name = r"C:\DataIAS\CompetitionCalendarSchoolSportFinans.csv"





df = pd.read_csv(file_name, error_bad_lines=False)

b'Skipping line 7105: expected 16 fields, saw 17\nSkipping line 7106: expected 16 fields, saw 17\nSkipping line 7107: expected 16 fields, saw 17\nSkipping line 7108: expected 16 fields, saw 17\nSkipping line 7109: expected 16 fields, saw 17\nSkipping line 7110: expected 16 fields, saw 17\nSkipping line 7111: expected 16 fields, saw 17\nSkipping line 7112: expected 16 fields, saw 17\n'


In [18]:
period = '2025'

In [19]:
#План
df['PlannedCost'] = df['PlannedCost'].replace(r'\s+', '', regex=True)
df['PlannedCost'] = df['PlannedCost'].replace('-', '0.0')
df['PlannedCost'] = df['PlannedCost'].str.replace(',', '.')
df['PlannedCost'] = df['PlannedCost'].astype('float')
# Факт
df['ActualCost'] = df['ActualCost'].replace(r'\s+', '', regex=True)
df['ActualCost'] = df['ActualCost'].replace('-', '0.0')
df['ActualCost'] = df['ActualCost'].str.replace(',', '.')
df['ActualCost'] = df['ActualCost'].astype('float')


In [20]:
df = df[[
    'CalendarSectionName1', 'Textbox1', 'NameSummary',
    'SportName', 'Summary', 'Appointment', 'Textbox117',
    'Count', 'Textbox9', 'FirmName', 'ExpenseTypeShortName',
    'PlannedCost', 'ActualCost'
]]


In [21]:
def parse_dates(date_str, period):
    # Определяем текущий и следующий годы для обработки
    current_year = period
    next_year = period + 1

    # Шаблоны для поиска одной или двух дат
    pattern_two_dates = r'(\d{2}\.\d{2}) (\d{2}\.\d{2})'
    pattern_one_date = r'(\d{2}\.\d{2})$'

    # Проверяем, соответствуют ли данные одному из шаблонов
    match_two = re.match(pattern_two_dates, date_str)
    match_one = re.match(pattern_one_date, date_str)

    if match_two:
        # Парсим две даты
        date_start_str, date_end_str = match_two.groups()
        date_start = datetime.strptime(date_start_str + f'.{current_year}', '%d.%m.%Y')
        date_end = datetime.strptime(date_end_str + f'.{current_year}', '%d.%m.%Y')
        
        # Если дата окончания меньше даты начала, то это переход на следующий год
        if date_end < date_start:
            date_end = datetime.strptime(date_end_str + f'.{next_year}', '%d.%m.%Y')
        
        return date_start, date_end

    elif match_one:
        # Если только одна дата, парсим и возвращаем её для обеих колонок
        date_start = datetime.strptime(match_one.group(1) + f'.{current_year}', '%d.%m.%Y')
        return date_start, date_start

    return None, None

def cor_dates(df, period):
    period = int(period)
    df[['Date_at', 'Date_of']] = df['Textbox117'].apply(lambda x: pd.Series(parse_dates(x, period)))
    #Считаем дни
    df['Count_day'] = (df['Date_of'] - df['Date_at']).dt.days + 1
    #Делаем квартал
    df['Quarter'] = pd.PeriodIndex(df['Date_of'], freq='Q')
    #Делаем месяц
    df['Month_m'] = pd.PeriodIndex(df['Date_of'], freq='M')
    return df

In [22]:
df = cor_dates(df, period)

In [23]:
def transform_record(record):
    # Используем регулярное выражение, чтобы оставить только нужную часть
    return re.sub(r'^\S+\s*', '', record)  # Убираем все, что перед первым пробелом

df['RazdelShool'] = df['CalendarSectionName1'].apply(transform_record)

In [24]:
from reports.mappings_report_fin import replace_df, colum_action, rename_columns


df = replace_df(df, 'RazdelShool')
df = colum_action(df)

In [11]:
def to_xlsx(df, period, name):
    name = str(name)
    if int(df.shape[0]) > 100:
        name_0 = r"C:\DataIAS\CalendarSchoolSportFinans\Calendar_"
    else:
        name_0 = r"C:\DataIAS\CalendarSchoolSportFinans\Analitika_"

    creation = datetime.today().strftime("%d_%m_%Y")
    tmp_df = name_0 + period + name + "_" + creation + ".xlsx"
    
    df.to_excel(tmp_df, index=False)
    print("Файл успешно сохранился")
    print(tmp_df)

In [12]:
to_xlsx(df, 'Пробный тест')

In [13]:
df

,CalendarSectionName1,RazdelShool,Textbox1,NameSummary,SportName,Summary,Appointment,Date_at,Date_of,Count_day,Quarter,Month_m,Textbox9,Count,FirmName,ExpenseTypeShortName,PlannedCost,ActualCost
0,6.1. ГКУ «ЦСТиСК» Москомспорта,ГКУ «ЦСТиСК» Москомспорта,43590,Учебно-тренировочное мероприятие по подготовке...,Фристайл,"могул, парный могул, юн-ки, юн-ры 14-19 лет","край. Пермский, г. Чусовой",2025-01-05,2025-01-17,13,2025Q1,2025-01,пров.,14,ГКУ «ЦСТиСК» Москомспорта,бюджет,1528.20,1528.2
1,6.1. ГКУ «ЦСТиСК» Москомспорта,ГКУ «ЦСТиСК» Москомспорта,68174,Чемпионат Москвы,Кёрлинг,"кёрлинг, командное, жен. от 14 лет и старше","Кёрлинг-центр «Москвич» (г. Москва, Волгоградс...",2025-01-05,2025-01-10,6,2025Q1,2025-01,пров.,12,РФСОО «ФКГМ»; ГКУ «ЦСТиСК» Москомспорта,бюджет,135.82,0.0
2,6.1. ГКУ «ЦСТиСК» Москомспорта,ГКУ «ЦСТиСК» Москомспорта,30141,Чемпионат Москвы,Спорт глухих (баскетбол),"баскетбол, командное, муж., жен.",Спортивный комплекс структурного подразделения...,2025-01-06,2025-01-12,7,2025Q1,2025-01,пров.,0,ГКУ «ЦСТиСК» Москомспорта; РСООИ «Федерация сп...,бюджет,177.72,0.0
3,6.1. ГКУ «ЦСТиСК» Москомспорта,ГКУ «ЦСТиСК» Москомспорта,82826,Учебно-тренировочное мероприятие по подготовке...,Фристайл,"ски-кросс, юниоры и юниорки 16-20 лет, девушки...","край. Пермский, г. Чусовой",2025-01-06,2025-01-15,10,2025Q1,2025-01,пров.,15,ГКУ «ЦСТиСК» Москомспорта,бюджет,1067.60,1067.6
4,6.1. ГКУ «ЦСТиСК» Москомспорта,ГКУ «ЦСТиСК» Москомспорта,2080,Учебно-тренировочное мероприятие по подготовке...,Тяжелая атлетика,"муж., жен.","Спортивная база «Чехов» (обл. Московская, р-н....",2025-01-09,2025-01-26,18,2025Q1,2025-01,пров.,0,ГКУ «ЦСТиСК» Москомспорта,бюджет,621.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7097,6.39.1. МСК «Жаворонки»,МСК «Жаворонки»,68516,Первенство ГБУ ДО «Московская академия танцева...,Акробатический рок-н-ролл,"А класс - микст, В класс - микст, формейшн, фо...",Многофункциональный спортивный комплекс «Жавор...,2025-04-27,2025-04-27,1,2025Q2,2025-04,пров.,1263,ГБУ ДО «Московская академия танцевального спор...,бюджет,60.00,60.0
7098,6.39.3. СБ «Олимпийские надежды»,СБ «Олимпийские надежды»,78127,Учебно-тренировочное мероприятие по подготовке...,Спорт глухих (баскетбол),"командное, девушки, юноши, юн-ки, юн-ры 7-21 л...",Спортивная база «Олимпийские надежды» (г. Моск...,2025-04-08,2025-04-16,9,2025Q2,2025-04,пров.,36,ГБУ ДО «Московская баскетбольная академия»,бюджет,573.30,0.0
7099,6.39.3. СБ «Олимпийские надежды»,СБ «Олимпийские надежды»,77453,Учебно-тренировочное мероприятие по подготовке...,Баскетбол,"баскетбол, командное, девушки, юноши 11-17 лет",Спортивная база «Олимпийские надежды» (г. Моск...,2025-05-01,2025-05-15,15,2025Q2,2025-05,пров.,0,ГБУ ДО «Московская баскетбольная академия»,бюджет,578.76,0.0
7100,6.39.3. СБ «Олимпийские надежды»,СБ «Олимпийские надежды»,77452,Учебно-тренировочное мероприятие по подготовке...,Баскетбол,"командное, девушки, юноши 11-17 лет",Спортивная база «Олимпийские надежды» (г. Моск...,2025-05-11,2025-05-31,21,2025Q2,2025-05,пров.,161,ГБУ ДО «Московская баскетбольная академия»,бюджет,1943.76,0.0
